In [61]:
# load all relevant packages and code

load('New_Falcon_stuff.sage')
load('CBD_stuff.sage')

import gc

from Entropy_stuff import *
from Multinomial import *
from Largelog import *
from Root_sum import approx_sum_of_roots
from Compact_Dictionary import *

In [ ]:
# Probability distributions are handled using dictionaries p, where the probability of sampling i is defined via
# p[i] (if i can be sampled)

B1_pdist={
    -1:1/4,
     0:2/4,
     1:1/4
}

In [ ]:
# The distribution class takes as input a probability distribution and optional value base, which denotes the
# entropy base. If unchanged, the latter is set to 2.

B1e=distribution(B1_pdist, base=e)
print(B1e)
print()
print()

B1=distribution(B1_pdist)
print(B1)
print()
print()

# When a distribution is defined via PMF instead of a dictionary of probabilities, we use

Falcon512=func_distribution('Falcon512', #Name of distribution
                            distribution_falcon, #PMF
                            [falcon512_sigma], #parameter set
                            ['sigma'] #parameter names
                           )
print(Falcon512)

In [ ]:
# If the input probability distribution is not normalized, the distribution class automatically normalizes it

B2_pdist={
    -2:1,
    -1:4,
     0:6,
     1:4,
     2:1,
}

B2=distribution(B2_pdist)
B2

In [ ]:
# This program comes with 5 predefined distributions: B1, B2, B3, Falcon512 (~D(4.06)) and Falcon1024 (~D(2.87)).
# Additionally, other centered binomial distributions with parameter eta can be created via CBD(eta)

B10=CBD(10)
B10

In [ ]:
# The normalized probability distribution of a distribution class object can be called with self.dist or self.d .
# The entropy is returned with self.entropy .

print("The Distribution of B(2) is defined through " + str(B2.dist)) # This object does not exist for func_distribution objects
print("The Entropy of B(2) is " + str(B2.entropy))
print("The Entropy of Falcon512 is " + str(Falcon512.entropy))

In [ ]:
# To see every unique probability value that exists in the distribution, call self.p (or self.log_p for
# their absolute log-values).

print("The non-zero probabilities in B(2) are " + str(B2.p)) # This object does not exist for func_distribution objects
print("Their absolute logs are " + str(B2.log_p)) # This object does not exist for func_distribution objects

# To see how often these occur, call self.m; the latter is ordered such that self.p[i] appears self.m[i] many times:

print("The amount of vectors with a certain probability are " + str(B2.m)) # This object does not exist for func_distribution objects

# To see all possible sampling values, sorted by their probability of sampling, call self.label

print("Specifically, the coordinates with a given probability are " + str(B2.label)) # This object does not exist for func_distribution objects

In [ ]:
# The size of range of possible probabilities is denoted with eta. Since we generally deal
# with distributions that are centered around 0, this usually coincides with the sampling range [-eta , ... , eta].
# The range can be called with self.range

print("The eta value of B(2) is " + str(B2.eta)) # This object does not exist for func_distribution objects
print("The sampling space of B(2) is " + str(B2.range)) # This object does not exist for func_distribution objects

In [ ]:
# To find the probability of sampling a certain i, call self.prob(i). Can also be called for elements not in
# the sampling range.

print("The probability to sample 2 in B(2) is " + str(B2.prob(2)))
print("The probability to sample 'Hello World' in B(2) is " + str(B2.prob("Hello World")))
print("The probability to sample 2 in Falcon512 is " + str(Falcon512.prob(2))) # func_distribution objects are
# not capable of finding the probabilities of events that don't lie in the sampling space.
print()

# To find the probability of sampling a certain vector v, call self.vec_prob(v)

print("Sampling [-2, 1, 0, -1] in B(2)^4 has probability " + str(B2.vec_prob([-2, 1, 0, -1])))
print("Sampling [-2, 1, 0, -1] in Falcon512^4 has probability " + str(Falcon512.vec_prob([-2, 1, 0, -1])))
print()


# A more compact way of representing a vector (and its unsigned permutations) is by counting
# how often a certain position/ probability occurs. For example, [-2, 1, 0, -1] can be represented
# by counting every 0, every +1, -1 and every +2, -2 and putting these weights in the list l = [1, 2, 1]
# to find the probability of a vector with only stating its weights can be done with self.compact_vec_prob(l)

print("Sampling a vector with 1 0, 2 +-1 and 1 +-2 (w/ fixed order) in B(2) has probability " + str(B2.compact_vec_prob([1, 2, 1])))
print()

# for func_distribution objects, the vector can be of any length

print("Sampling a vector with 1 0 and 1 +-101 (w/ fixed order) in Falcon512 has probability " + str(Falcon512.compact_vec_prob([1] + 100*[0] + [1])))
print()


# each probability function has optional input f, which, if set to true, converts the output to float:

print("Sampling a vector with 1 0 and 2 +-1 (w/ fixed order) in B(2) has probability " + str(B2.compact_vec_prob([1, 2], f = True)) + " (or " + str(B2.compact_vec_prob([1, 2], f = False)) + ")")

In [ ]:
# The calculation of (expected) runtimes requires the ability to build compact dictionaries.
# To create a compact dictionary for dimension n, call self.comp_dic(n):

print("The compact dictionary of B(2)^3 consists of a " + str(B2.comp_dic(3))) # Impossible for func_distribution objects,
# as compact dictionaries are infinitely large
print()
# Note that these dictionaries are of size O(n^eta), which can be very large for wide distributions
# (like D(4.06) or D(2.87)). To combat this, we can create partial compact dictionaries that only contain
# vectors above a certain probability threshold, say 2^(-H(.)n-offset) for entropy H(.) and some constant offset.
# To create such an partial compact dictionary, call self.par_comp_dic(n, offset):

print("One partial compact dictionary of Falcon512^3 consists of a " + str(Falcon512.par_comp_dic(3,offset=1)))
print()
print("Another partial compact dictionary of Falcon512^3 consists of a " + str(Falcon512.par_comp_dic(3,offset=0)))
print()
print("Another partial compact dictionary of Falcon512^3 consists of a " + str(Falcon512.par_comp_dic(3,offset=-1)))

# If a partial compact dictionary has been calculated previously for parameters (n, offset) and the function is
# called again for (n, offset') where offset' < offset, the former list can be reused to calculate the compact dictionary faster. 
# If offset' > offset, we have to restart the whole computation process. We can not use the already existing partial
# compact dictionary to make this process faster.

In [ ]:
# distribution class objects contain a pointer to all their peviously created compact dictionaries. These
# pointers can be found in the dictionary self.comp_dics:

print("We have created the following compact dictionaries for B(2): " + str(B2.comp_dics))
print("We have created the following compact dictionaries for D(4.06): " + str(Falcon512.comp_dics))

# To call a specific (partial) compact dictionary for dimension n, call self.comp_dic_list(n):

print("We have created the following compact dictionary for D(4.06)^3: " + str(Falcon512.comp_dic_list(3)))

# If said compact dictionary has not yet been computed, this returns an empty compact dictionary instead:

print("We have created the following compact dictionary for D(4.06)^(2^200): " + str(Falcon512.comp_dic_list(2**200)))

In [ ]:
# Compact dictionaries are their own class. To access the actual dictionary, call self_cd.dic

B2.par_comp_dic(4)
ex_cd=B2.comp_dic_list(4)
ex_fcd=Falcon512.comp_dic_list(3)
print("The compact dictionary for B(2)^4 looks like this: " + str(ex_cd.dic))
print()



# every item in self_cd.dic is a list that contains 4 items: the actual weight distribution, the sampling distribution
# for a vector with said distribution, the amount of vectors that have this unsigned weight distribution and the 
# amount of vectors from previous (i.e. more likely) entries.

print("There are a total of " + str(ex_cd.dic[2][2]) + " vectors with weight distribution "
      + str(ex_cd.dic[2][0]) + ". Their sampling probability is "+ str(ex_cd.dic[2][1]) + 
      " (each). There are " + str(ex_cd.dic[2][3]) + " vectors that are at least as likely as these vectors.")



In [ ]:
# To retreive the amount of vectors that are represented with the stored partial dictionary, call self_cd.count
# (or logcount() if you need the log of count)

print("there are " + str(ex_cd.count) + " vectors represented through the partial compact dictionary of B(2)^4. "
      "As a power of 2, that is 2^" + str(ex_cd.logcount()) + ".")
print("there are " + str(ex_fcd.count) + " vectors represented through the partial compact dictionary of Falcon512^3. "
      "As a power of 2, that is 2^" + str(ex_fcd.logcount()) + ".")
print()

# The cumulative sampling probability of all these vectors can be returned with self_cd.p

print("The probability of sampling an element from the partial compact dictionary of B(2)^4 is " + str(ex_cd.p) + ".")
print("The probability of sampling an element from the partial compact dictionary of Falcon512^3 is " + str(ex_fcd.p) + ".")

In [ ]:
# The value for offset from function call self.par_comp_dic(n, offset) is stored in self_cd.offset

print("Every element in the partial compact dictionary of B(2)^4 has sampling probability at least 2^(-H(B(2))*3 -"
      +  str(ex_cd.offset) + ").") 
print("Every element in the partial compact dictionary of Falcon512^3 has sampling probability at least 2^(-H(Falcon512)*4 -"
      +  str(ex_fcd.offset) + ").") 
print()

# The empty compact dictionary has value c set to -inf. Compact dictionaries created with calling self.comp_dic(n)
# have their value of c set to -(H(.)-max(self.log_p))n+1

print("Elements from the empty compact dictionary have sampling probability 2^(-H(chi)*n +" + str(-empty_comp_dic.offset) + ").")
print("Elements from the non-partial, compact dictionary of B(2)^3 have sampling probability at least 2^(-H(B(2))*3 -"
       + str(B2.comp_dic_list(3).offset) + ").")

In [ ]:
# To create csv-style tables containing raw data for n in range [low_n, high_n], call self.raw_data():

B2.raw_data(1,3)
print()
Falcon512.raw_data(4,6)

# the column heads of the csv table are
# n: vector dimension n
# p: probability that a randomly sampled vector has sampling probability 2^(-H(.)n-offset)
# runtime: amount of vectors that satisfy the above condition
# Eclassic: expected runtime of running AbortedKeyGuess on that set of vectors
# Equantum: expected runtime of running Montanaro's algorithm on the very set of vectors

In [ ]:
# By default, we set offset to 0. If another calue for offset is required, offset can optionally be altered:

B2.raw_data(1,3, offset=3)
print()
Falcon512.raw_data(4,6, offset=3)
print()

In [ ]:
# If not every element from that range is required, the step size can be increased with the optional step command:

B2.raw_data(1,5, step=2)
print()
Falcon512.raw_data(4,8, step=2)

In [ ]:
# Since we do not need the complete compact dictionary except when we compare the expected runtimes of KeyGuess
# and AbortedKeyGuess, we omit the expected runtime of KeyGuess unless specifically asked for. This can be done
# with the optional command aborts = False:

B2.raw_data(1,3, aborts = False) # This object does not exist for func_distribution objects
print()

# The last function call has an additional column that contains the expected runtime of KeyGuess with column head
# Enoabort: Expected runtime of KeyGuess w/o aborts

In [ ]:
# If the compact dictionaries are no longer required after the csv table is computed, the optional command
# delete_after can be set to true to immediately delete the compact dictionaries:

print(list(B2.comp_dics))
B2.raw_data(1,3, delete_after = True)
print(list(B2.comp_dics))
print()


print(list(Falcon512.comp_dics))
Falcon512.raw_data(4,6, delete_after = True)
print(list(Falcon512.comp_dics))

# Note how the last call of B2.comp_dics does not contain the keys n = 1, 2, 3.

In [ ]:
# Note that the previous method returns the analysis for KeyGuess for ALL vectors represented in the  partial compact
# dictionary. In general, we do not assume that the partial compact dictionary represents exactly 2^(H(chi)*n) many
# vectors. To get the results for exactly 2^(H(chi)*n) many vectors, you have to run raw_data_veccount instead:

B2.raw_data_veccount(1,3)
print()
Falcon512.raw_data_veccount(4,6)

In [ ]:
# To get the results for sampling 2^(H(chi)*n+border_offset) many vectors instead, call
# dist.raw_data_veccount(low, high, border_offset=border_offset instead:)

B2.raw_data_veccount(1,3, border_offset=-1)
print()
Falcon512.raw_data_veccount(4,6, border_offset=1)

In [ ]:
# The routine GetKey returns the i-th most likely vector (starting at 0, sorted lexicographically and by order of signs).
# It requires the precomputation of the compact dictionary first

print("The 16,050,000-th most likely key of B(3)^10 is " + str(B3.GetKey(16049999,10)))
print()
print()

# To get an ordered list of consecutive vectors , call GetKeys instead

print("The 100,000 to 100,010-th most likely keys of B(3)^10 are")
print()
print(B3.GetKeys(99999,100009,10))